<a href="https://colab.research.google.com/github/IshaqKHATTAK/Movie-recomeneded-system-usinig-streamlit-machine-learning/blob/main/movie_reconded_systemipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import ast

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd /content/drive/MyDrive/movie data

[Errno 2] No such file or directory: '/content/drive/MyDrive/movie data'
/content


In [7]:
#!unzip /content/drive/MyDrive/movie_data/tmdb_5000_movies.csv.zip
#run once

In [8]:
#!unzip /content/drive/MyDrive/movie_data/tmdb_5000_credits.csv.zip

In [9]:
movies = pd.read_csv('/content/drive/MyDrive/movie_data/tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [10]:
credits = pd.read_csv('/content/drive/MyDrive/movie_data/tmdb_5000_credits.csv')
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [11]:
# merge the dataset credit and movie on the basis of title column
df = pd.merge(movies, credits, on="title")
df.shape

(4809, 23)

In [12]:
#we need to remove some of the columns because these are not important for recomendation system that we are
#implementing.
#budgest ;; we will not keep that with us becoz is does not matter that if i like movie it should be based on the budges
#genres: should be keep
#homepage: no need
#id: use for fetching movie (keep it)
#keywords: keep it
#original_language this column is almost partial so no need to keep it with us


In [13]:
movies = df[['genres','id','keywords','title','overview','cast','crew']]


movies.shape

(4809, 7)

In [14]:
movies.head(5)

,genres,id,keywords,title,overview,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [15]:
movies.isna().sum()

genres      0
id          0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [16]:
movies = movies.drop_duplicates()

In [17]:
import ast

def to_form(text):
  '''
  input text:  "[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]"
  output: '['Action', 'Adventure', 'Fantasy', 'Science Fiction']
  '''
  li = []
  for i in ast.literal_eval(text):
    li.append(i['name'])
  return li



In [18]:
movies['genres'] = movies['genres'].apply(to_form)

In [19]:
movies['keywords'] = movies['keywords'].apply(to_form)

In [20]:
#in this function i jsut need to extract the first theree dictionay based on importance
def convert_cast(text):
  a = ast.literal_eval(text) #the ast.literal is to convert the string into the list
  li = []
  for i in a[:3]:
    li.append(i['name'])
  return li

In [21]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [22]:
#function to fetch the name where the job is director in the crew column
def convert_crew(text):
  li = []
  a = ast.literal_eval(text)
  for i in a:
    if i['job'] == 'Director':
      li.append(i['name'])
  return li

In [23]:
movies['crew'] = movies['crew'].apply(convert_crew)

In [24]:
movies['overview'] = movies['overview'].apply(lambda x:str(x).split())
#convert the string into list form.

In [25]:
movies.head(4)

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...",The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]


In [26]:
#now we are set with the data
#but stril we have a problem of name seperation which leads to modle confustion like
#muhmmad ishaq and muhammad zeshan the modle will confuse weather you need muhammad ishaq or zershan
#becuze the token all both will be same (first token(muhammad))
#the solutin is we need to remove the space in between name

In [27]:
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ','') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ','') for i in x])
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ','') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ','') for i in x])

In [28]:
movies.head(3)

,genres,id,keywords,title,overview,cast,crew
0,"[Action, Adventure, Fantasy, ScienceFiction]",19995,"[cultureclash, future, spacewar, spacecolony, ...",Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,"[Adventure, Fantasy, Action]",285,"[ocean, drugabuse, exoticisland, eastindiatrad...",Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,"[Action, Adventure, Crime]",206647,"[spy, basedonnovel, secretagent, sequel, mi6, ...",Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]


In [29]:
#now i want to concate all the colums and get one new column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [30]:
new_df = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [31]:
new_df

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [32]:
new_df['tags']  = new_df['tags'].apply(lambda x: ' '.join(x))

In [33]:
new_df['tags'] = new_df['tags'].apply(lambda x : x.lower())

In [34]:
new_df.head(2)

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."


In [35]:
#now to find the similarity based on the movies title we need to find the similarity of that movie with other
#movie so for that we need to convert movie into vectors and then find similarity matrix (find angel theta)
#we do not use (x1-x2)^2 bcoz that is not that good on large diamentions.


In [36]:
# remove stop word and stemming
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')


def preprocessing(text):
  ps = PorterStemmer()
  tok = word_tokenize(text)
  sw = [word for word in tok if word not in stopwords.words('english')]
  final = [ps.stem(word) for word in sw]
  return final
#preprocessing('this is ishaq from calling fro')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [37]:
new_df['tags']  = new_df['tags'].apply(preprocessing)
new_df.head(3)

,id,title,tags
0,19995,Avatar,"[22nd, centuri, ,, parapleg, marin, dispatch, ..."
1,285,Pirates of the Caribbean: At World's End,"[captain, barbossa, ,, long, believ, dead, ,, ..."
2,206647,Spectre,"[cryptic, messag, bond, ’, past, send, trail, ..."


In [38]:
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))
new_df.head(2)

,id,title,tags
0,19995,Avatar,"22nd centuri , parapleg marin dispatch moon pa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa , long believ dead , come bac..."


In [39]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000)

vectors = cv.fit_transform(new_df['tags']).toarray()

In [40]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

distances = cosine_similarity(vectors)

In [42]:
def recomend(movie_title):
  movie_indx = new_df[new_df['title'] == movie_title].index[0]
  sorted_list = sorted(list(enumerate(distances[0])),reverse=True,key = lambda x: x[1])
  for indx in sorted_list[1:6]:
    print(new_df.iloc[indx[0]].title)

recomend('Avatar')

Falcon Rising
Aliens vs Predator: Requiem
Aliens
Titan A.E.
Independence Day


In [43]:
import pickle
pickle.dump(new_df,open('movie_list.pkl','wb'))
pickle.dump(distances,open('similarity.pkl','wb'))

In [44]:
pickle.dump(new_df.to_dict(), open('movie_list_dict.pkl','wb'))